In [6]:
import requests
from requests.exceptions import ConnectionError
requests.packages.urllib3.disable_warnings()
import time # time.sleep(seconds)
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
from time import sleep

In [3]:
# 金額轉換
common_used_numerals = { '〇' : 0, '一' : 1, '二' : 2, '三' : 3, '四' : 4, '五' : 5, '六' : 6, '七' : 7, '八' : 8, '九' : 9, '零' : 0, '壹' : 1, '貳' : 2, '參' : 3, '叄' : 3, '肆' : 4, '伍' : 5, '陸' : 6, '柒' : 7, '捌' : 8, '玖' : 9, '貮' : 2, '兩' : 2,'十' : 10, '拾' : 10, '百' : 100, '佰' : 100, '千' : 1000, '仟' : 1000, '萬' : 10000, '萬' : 10000, '億' : 100000000, '億' : 100000000, '兆' : 1000000000000,}

def chinese2digits(uchars_chinese):
    string_list = []
    for i in uchars_chinese:
        string_list.append(i)
    if string_list[0] == '十'or string_list[0] =='拾':
        uchars_chinese = '一'+uchars_chinese
    total = 0
    r = 1     # 個、十、百、千、萬         
    for i in range(len(uchars_chinese)-1, -1, -1): # 從最後一個字開始處理
        val = common_used_numerals.get(uchars_chinese[i])
        if val >= 10 and i == 0:  
            if val > r:
                r = val
                total = total + val
            else:
                r = r * val

        elif val >= 10:
            if val > r:
                r = val
            elif val <= r and r >= 100000:
                r = int(r * (val/(val/10))) # 處理特殊值: 一百三十萬
            else:
                r = r * val

        else:
            total = total + r * val

    return total

def digitsandchinese(xwan):#當金額是數字加上國字萬，如5萬、2仟
    ans=[]
    wan=0
    thd=0
    hud=0
    ten=0
    x=0
    for i in range(len(xwan)):
        if xwan[i]=='萬':
            digits=int("".join(ans))
            wan=digits*10000
            ans=[]
        elif xwan[i]=='仟' or xwan[i]=='千':
            digits=int("".join(ans))
            thd=digits*1000
            ans=[]
        elif xwan[i]=='佰' or xwan[i]=='百':
            digits=int("".join(ans))
            hud=digits*100
            ans=[]
        elif xwan[i]=='拾' or xwan[i]=='十':
            digits=int("".join(ans))
            ten=digits*10
            x=int(xwan[int(i+1)])
        else:
            ans.append(xwan[i])
    ans=wan+thd+hud+ten+x
    return ans

In [4]:
#打開分頁、獲得判決書列表頁面url
crime = input('裁判案由 :')
place = input('法院 :')
browser = webdriver.Chrome(executable_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
url="https://law.judicial.gov.tw/FJUD/default_AD.aspx"
browser.get(url)

browser.find_element_by_name('jud_sys').click()

content = browser.find_element_by_name('jud_kw')
content.send_keys(crime)

#place_list = ['臺灣台北地方法院', '臺灣新北地方法院', 臺灣台中地方法院, 臺灣台南地方法院, 臺灣高雄地方法院, 臺灣花蓮地方法院]
browser.find_element_by_name('ctl00$cp_content$btnQry').click()
TPD = browser.find_element_by_partial_link_text(place)
ActionChains(browser).move_to_element(TPD).key_down(Keys.COMMAND).key_down(Keys.SHIFT).click(TPD).key_up(Keys.CONTROL).key_up(Keys.SHIFT).perform()
handles = browser.window_handles#取得所有分頁
browser.switch_to.window(handles[1])#換到新分頁
newurl = browser.current_url

裁判案由 :公然侮辱
法院 :臺灣臺北地方法院


In [9]:
#爬出所有判決
article_num = 0 # article index
content_dic = {}
all_link = {}

while article_num < 20:
    res = requests.get(newurl, verify=False) # url for agenda
    soup=BeautifulSoup(res.text, 'html.parser')

    link_list = []
    for link in soup.find_all('a'):
        href = link.get('href')
        link_list.append(href)
        
    if article_num < 20:
        url_list = link_list[7:27] # url for articles + next_page + final+page
    else:
        url_list = link_list[9:29]
        
    next_page = "https://law.judicial.gov.tw" + link_list[-2] + '&ot=in'

    for article_link in url_list:
        article_num += 1
        all_link.update({str(article_num) : article_link})
        print(article_num, end = ' ')

        try:
            article = requests.get("https://law.judicial.gov.tw/FJUD/" + article_link, verify=False)
            soup=BeautifulSoup(article.text, 'html.parser')
            body = soup.find('body')
            # clean up the content of article
            main_article = body.find('div', {'class':'text-pre text-pre-in'}).text.replace(' ','').replace('\n','').replace('\r','').replace('，','').replace('、','')
        except ConnectionError:
            main_article = 'No response'
        except:
            main_article = 'No response'

        # pattern for regular expression
        regex_word = re.compile('「\w+」', re.IGNORECASE)
        regex_money = re.compile('被告應給付原告新臺幣\w+元' or '被告應給付原告新台幣\w+元', re.IGNORECASE)
        
        # 找出僅包含公然侮辱的判決書
        main_article = main_article.replace('公然侮辱','OOOO').replace('妨害名譽','OOOO').replace('誹謗','OO').replace('等罪嫌','等罪').replace('罪惡','OO').replace('罪嫌','OO').replace('罪名','OO').replace('認罪','OO').replace('O罪','OO').replace('有罪','OO').replace('無罪','OO').replace('罪責','OO').replace('犯罪','OO').replace(' 罪','OO')

        if '罪' not in main_article:#限定於只有公然侮辱罪之案件
            out = regex_word.findall(main_article)
            money = regex_money.findall(main_article)
            if money == [] or out == []: # there is no fine
                money.append('N/A')
            else:
                # transform the value of money
                if len(money) > 1:
                    money = 'N/A'
                else: 
                    money = str(money).replace("被告應給付原告新臺幣","").replace("被告應給付原告新台幣","").replace("新臺幣","").replace("新台幣","").replace("臺幣","").replace("台幣","").replace("元","").replace("[","").replace("]","").replace("'","")
                    try:#判讀是哪種寫法
                        money = chinese2digits(money)
                    except TypeError:
                        try:
                            first_letter=int(money[0])#如果第一個字可以變成int表示是以數字加國字形式出現
                            money = digitsandchinese(money)
                        except:
                            money = 'N/A'
                    except:
                        money = 'N/A'
                        
                if money != 'N/A':
                    content_dic.update({str(article_num): [out, money]}) # save as a dictionary
                else:
                    continue
        else:
            continue

    newurl = next_page # change page
    sleep(1)
    
print()
print('All Done')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
All Done


In [10]:
# search engine
s1 =input('髒話(完全匹配請加「」) : ')
a = 0
total=0
num=0

for art_num in content_dic:
    for word in content_dic[art_num][0]:
        if s1 in word:
            a = 1
        else:
            continue
    if a > 0:
        print('Article:', art_num, 'Fine:',content_dic[art_num][1])
        total=total+int(content_dic[art_num][1])
        num+=1
        a = 0
    else:
        continue

if num>0:
    aver=str(total/num)
    print('平均='+aver)
else:
    print('未查有判決')

髒話(完全匹配請加「」) : 幹
Article: 4 Fine: 60000
Article: 6 Fine: 10000
平均=35000.0


In [11]:
#open the article
page = input('文章序號 :')
browser = webdriver.Chrome(executable_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
browser.get('https://law.judicial.gov.tw/FJUD/' + all_link[page])

文章序號 :4
